In [ ]:
from sentence_transformers import SentenceTransformer
from typing import List
import numpy as np
import nmslib

# Loading the transformer model
# SentenceTransformer uses transformer models (like BERT, RoBERTa, etc.)
# It's designed to produce sentence embeddings, i.e. a semantic representation for an entire sentence
model = SentenceTransformer('all-MiniLM-L6-v2')

def create_index(embeddings):
    """
    This function creates an nmslib index from the embeddings of the sentences.

    :param embeddings: The sentence embeddings
    :return: An nmslib index
    """
    # Initialize a new index
    # method='hnsw': we use HNSW (Hierarchical Navigable Small World), which is a state-of-the-art
    # method for nearest neighbor search
    # space='cosinesimil': we use cosine similarity as the similarity metric
    index = nmslib.init(method='hnsw', space='cosinesimil')

    # Add the sentence embeddings to the index
    # The embeddings are efficiently stored so that the nearest neighbors to a query can be quickly computed
    index.addDataPointBatch(embeddings)
    
    # Create the index
    # 'post': 2: It determines the amount of work that should be performed during the index construction
    index.createIndex({'post': 2}, print_progress=True)
    return index

def search(query: str, index, sentence_list, top_k: int = 5):
    """
    This function converts a query to an embedding and finds the top_k most similar sentences.

    :param query: The input query
    :param index: The nmslib index
    :param sentence_list: The list of sentences
    :param top_k: The number of nearest neighbors
    :return: The top_k most similar sentences to the query
    """
    # Convert the query to an embedding
    query_embedding = model.encode([query])

    # Search for the top_k most similar sentences to the query
    # knnQuery returns two lists: a list of indices of the nearest neighbors and a list of the corresponding distances
    ids, distances = index.knnQuery(query_embedding, k=top_k)
    
    # Map the list of indices to the actual sentences and return it
    return [sentence_list[i] for i in ids]

# Assume we have a list of sentences (conversational history)
conversational_history = [
    "Hello, how are you?",
    "I'm fine, thank you!",
    "What's your favorite color?",
    "I love blue.",
    "Do you like ice cream?",
    "Yes, especially vanilla flavor.",
    # more sentences...
]

# Transform the sentences to embeddings
# Each sentence is mapped to a high-dimensional vector
# Semantically similar sentences will be close in this vector space
sentence_embeddings = model.encode(conversational_history)

# Convert the list of sentence embeddings to a numpy array
# This is necessary because nmslib expects an array-like structure
sentence_embeddings_np = np.array(sentence_embeddings)

# Create the search index from the sentence embeddings
# The index can be used to efficiently find the sentences most similar to a query
search_index = create_index(sentence_embeddings_np)

# Search for the most similar sentences to a query
# The function will return the sentences most similar to the query
query = "Who likes dessert?"
print(search(query, search_index, conversational_history))
